In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

In [2]:
rf=pd.read_csv("Fraud_check.csv")

In [3]:
rf

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


# EDA

In [4]:
rf.isna().sum()

Undergrad          0
Marital.Status     0
Taxable.Income     0
City.Population    0
Work.Experience    0
Urban              0
dtype: int64

In [6]:
rf[rf['Taxable.Income']<=30000]

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
10,NO,Single,29732,102602,19,YES
12,NO,Divorced,11794,148033,14,YES
16,YES,Divorced,24987,34551,29,NO
19,NO,Single,10987,126953,30,YES
21,YES,Divorced,14310,29106,7,YES
...,...,...,...,...,...,...
544,NO,Single,29916,133877,21,YES
557,YES,Married,12810,157421,10,NO
567,YES,Single,16316,129912,30,NO
575,NO,Divorced,10735,95975,13,YES


In [7]:
gp=['Risky','Good']
rf['Fraudcheck']=pd.cut(rf['Taxable.Income'],bins=[0,30000,1000000],labels=gp)

In [8]:
for i in range(0,len(rf['Taxable.Income'])):
    if rf['Taxable.Income'][i]<=30000:
        rf['Fraudcheck'][i] = 'Risky'
    else:
        rf['Fraudcheck'][i]='Good'

In [10]:
rf.head(7)

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,Fraudcheck
0,NO,Single,68833,50047,10,YES,Good
1,YES,Divorced,33700,134075,18,YES,Good
2,NO,Married,36925,160205,30,YES,Good
3,YES,Single,50190,193264,15,YES,Good
4,NO,Married,81002,27533,28,NO,Good
5,NO,Divorced,33329,116382,0,NO,Good
6,NO,Divorced,83357,80890,8,YES,Good


In [11]:
rf['Taxable.Income'].describe()

count      600.000000
mean     55208.375000
std      26204.827597
min      10003.000000
25%      32871.500000
50%      55074.500000
75%      78611.750000
max      99619.000000
Name: Taxable.Income, dtype: float64

In [12]:
rf['Fraudcheck'].value_counts()

Good     476
Risky    124
Name: Fraudcheck, dtype: int64

In [15]:
rf.head(0)

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,Fraudcheck


In [60]:
rf['Fraudcheck']=rf['Fraudcheck'].replace(['Risky','Good'],[0,1])
rf['Urban']=rf['Urban'].replace(['YES','NO'],[1,0])
rf['Undergrad']=rf['Undergrad'].replace(['YES','NO'],[1,0])
rf['Marital.Status']=rf['Marital.Status'].replace(['Single','Divorced','Married'],[0,1,2])

In [61]:
rf

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,Fraudcheck
0,0,0,68833,50047,10,1,1
1,1,2,33700,134075,18,1,1
2,0,1,36925,160205,30,1,1
3,1,0,50190,193264,15,1,1
4,0,1,81002,27533,28,0,1
...,...,...,...,...,...,...,...
595,1,2,76340,39492,7,1,1
596,1,2,69967,55369,2,1,1
597,0,2,47334,154058,0,1,1
598,1,1,98592,180083,17,0,1


In [62]:
rf['Fraudcheck'].value_counts()

1    476
0    124
Name: Fraudcheck, dtype: int64

In [63]:
rf['Marital.Status'].value_counts()

0    217
1    194
2    189
Name: Marital.Status, dtype: int64

In [64]:
rf['Urban'].value_counts()

1    302
0    298
Name: Urban, dtype: int64

# SPLITTING THE DATA

In [65]:
rf.head()

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,Fraudcheck
0,0,0,68833,50047,10,1,1
1,1,2,33700,134075,18,1,1
2,0,1,36925,160205,30,1,1
3,1,0,50190,193264,15,1,1
4,0,1,81002,27533,28,0,1


In [85]:
rf.columns

Index(['Undergrad', 'Marital.Status', 'Taxable.Income', 'City.Population',
       'Work.Experience', 'Urban', 'Fraudcheck'],
      dtype='object')

In [86]:
rf1=rf[['Undergrad','Marital.Status','City.Population','Work.Experience','Urban','Fraudcheck']]

In [87]:
rf1.head()

,Undergrad,Marital.Status,City.Population,Work.Experience,Urban,Fraudcheck
0,0,0,50047,10,1,1
1,1,2,134075,18,1,1
2,0,1,160205,30,1,1
3,1,0,193264,15,1,1
4,0,1,27533,28,0,1


In [88]:
X=rf1.iloc[:,:-1]
Y=rf1.iloc[:,-1]

In [89]:
X.head()

,Undergrad,Marital.Status,City.Population,Work.Experience,Urban
0,0,0,50047,10,1
1,1,2,134075,18,1
2,0,1,160205,30,1
3,1,0,193264,15,1
4,0,1,27533,28,0


In [90]:
Y.head()

0    1
1    1
2    1
3    1
4    1
Name: Fraudcheck, dtype: category
Categories (2, int64): [0 < 1]

In [91]:
rf1.columns

Index(['Undergrad', 'Marital.Status', 'City.Population', 'Work.Experience',
       'Urban', 'Fraudcheck'],
      dtype='object')

# SPLITTING DATA INTO TRAIN AND TEST

In [92]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=10)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((480, 5), (120, 5), (480,), (120,))

In [93]:
x_train

,Undergrad,Marital.Status,City.Population,Work.Experience,Urban
284,0,2,188383,22,1
423,0,1,80249,19,1
525,1,0,75307,23,0
519,1,0,162102,25,1
12,0,2,148033,14,1
...,...,...,...,...,...
369,0,1,89122,28,0
320,1,0,78897,11,1
527,0,1,180274,27,0
125,1,2,155669,11,0


In [94]:
y_train

284    0
423    1
525    1
519    1
12     0
      ..
369    1
320    1
527    1
125    1
265    0
Name: Fraudcheck, Length: 480, dtype: category
Categories (2, int64): [0 < 1]

# GRID SEARCH CV TO FIND BEST HYEPRPARAMETER

In [95]:
from sklearn.model_selection import GridSearchCV

In [96]:
model=RandomForestClassifier()
parameters={
    
    "n_estimators":[130,140,150,160],
    "max_features":[2,3,4,5]
}

In [97]:
G_dt=GridSearchCV(model,parameters,cv=10)

In [98]:
G_dt.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_features': [2, 3, 4, 5],
                         'n_estimators': [130, 140, 150, 160]})

In [99]:
G_dt.best_score_

0.7666666666666666

In [100]:
G_dt.best_params_

{'max_features': 3, 'n_estimators': 160}

In [101]:
model2=RandomForestClassifier(max_features=2, n_estimators=130)

In [102]:
model2.fit(x_train,y_train)

RandomForestClassifier(max_features=2, n_estimators=130)

In [103]:
print(classification_report(y_test,model2.predict(x_test)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        26
           1       0.77      0.90      0.83        94

    accuracy                           0.71       120
   macro avg       0.38      0.45      0.41       120
weighted avg       0.60      0.71      0.65       120



In [104]:
model2.feature_importances_

array([0.03606342, 0.06441502, 0.53484515, 0.33062323, 0.03405318])